# Langevin
##  Theory
- integration schemes that include a thermostat
- full algorithms to simulate molecular dynamics in the N V T ensemble
- stochastic dynamics, based on Langevin dynamics
$$
dp = M^{-1}v \,\text{d}t\\
dv = -\nabla \phi(p)\,\text{d}t- \gamma v \,\text{d}t + \sigma M^{1/2}\,\text{d}W
$$ 
     - the first part of the equation is equal to Newtonian dynamics
     - and the function of the last two terms is to act as a thermostat (friction + noise)

whereby:
- $W = W (t)$ is a vector of 3N independent Wiener processes ? => results in the matrix of noise intensities and a vector of uncorrelated Gaussian random numbers $R_t$, 
* $\gamma > 0$ is a free (scalar) parameter the isotopic friction constant which couple the system to the bath (damping parameter), 
* choosing $ \sigma = \sqrt{2\gamma \beta ^{-1}}$ it is possible to show that the unique probability distribution sampled by the dynamics is the canonical (Gibbs-Boltzmann) density

integration by discretizing in time using a second-order numerical method  according to 
$$\hat{L}*= L*_{\text{LD}}+ \delta t ^2 L*_2 + O(\delta t^3)$$
instead of Taylor series expansion

for the BAOAB method the Langevin dynamics are breaked into three pieces
$$
\left[ \begin{matrix}dp\\dv\end{matrix}\right]= \left[ \begin{matrix}M^{-1}v\\0\end{matrix}\right]\text{d}t+\left[ \begin{matrix}0\\-\nabla\phi(p)\end{matrix}\right]\text{d}t
+\left[ \begin{matrix}0\\-\gamma v \text{d}t + \sigma M^{1/2}\text{d}W\end{matrix}\right]$$

- firts part is labelled A, second as B and third O
O piece is associated with an Ornstein-Uhlenbeck equation with “exact” solution:
$$v(t) = e^{-\gamma t} v(0)+ \frac{\sigma}{\sqrt{2\gamma}}\sqrt{1-e^{-2\gamma t}}M^{1/2}R_t$$
where $R_t$ is a vector of uncorrelated noise processes

- the sequence is given through BAOAB
- to ensure the method is symmetric  all “A” and “B” parts in BAOAB are integrated for a half timestep 

### Advantage
-
-


## Algorithm
- choice between open and closed system in potential
- pbc defined in algoritm 
- pbc and closed system condition do not work together 


1. defining parameters
      - number of time steps
      - number of particles
      - number of dimension
2. prepering the time iteration
      - creating positiion, velocity and acceleration container
      - initialization by adding the start configuration
3. time iteration
      - define iteraration scheme (loop p=t-1)
      - langevin approximation
          - B
          - A
          - O
          - A
          - B
      - writing the output array
4. returning position_matrix, velocity_matrix, acceleration_matrix


## Goals
- timing 
- testing 
- visu
- tempering


In [ ]:
%matplotlib notebook
import numpy as np
import timeit
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
%load_ext line_profiler

### potential
- defining forces
- defining system
    - open system
    - closed system

In [ ]:
def potential_gradient(p, boxsize, k=0):
    return k*p

In [ ]:
def potential_gradient(p, boxsize, k=1):
    p_max, p_min = box[0], box[1]
    V_r = (p - (p_max - p_max/4))*(p>=p_max)
    V_l = (p - (p_min - p_min/4))*(p<=p_min)
    V = V_r+ V_l 
    return V  

### algorithm

In [ ]:
def langevin(potential_gradient, position_init, velocity_init, mass, total_time, time_step, damping, beta, temp, boxsize, pbc=True):
    
    """
    This function realise the integration scheme of Langevin dynamics with the BAOAB-Algorithm. 
    This scheme includes a thermostat thus solving ODEs leads to a MD-Simulation in the NVT-ensemble.
    
    Arguments:
        potential_gradient (function): computes potential gradient for particle-positions 
        position_init (numpy.ndarray(n, dim)): initial configuration in dim dimensions
        velocity_init (numpy.ndarray(n, dim)): initial velocity in dim dimensions
        mass (numpy.ndarray(n)): mass of each particle
        total_time (int): total time of integration
        time_step (float): step size for integration 
        damping (float): isotopic friction constant (couple the system to the bath), zero for not coupled
        beta (float): inverse temperature
        temp (float): tempering parameter, choose positive value to warm up the system,
                      negative to cooling for each time step and zero no tempering
        pbc (boolean): periodic boundary conditions (True) or not periodic (False)
        
    Returns:
        position_matrix (numpy.ndarray(size, n, dim)): configuraiton trajectory
        velocity_matrix (numpy.ndarray(size, n, dim)): velocity trajectory
        acceleration_matrix (numpy.ndarray(size, n, dim)): acceleration trajectory
       
    """
    size = int(total_time/time_step)    
    n = len(position_init)              
    dim = position_init.shape[-1]       
    m = mass
    p_min, p_max = boxsize[0], boxsize[1]
    position_matrix, velocity_matrix, acceleration_matrix = np.zeros((size, n, dim)), np.zeros((size, n, dim)), np.zeros((size, n, dim))
    position_matrix[0], velocity_matrix[0], acceleration_matrix[0] = position_init, velocity_init, potential_gradient(position_init, boxsize)
    R_t = np.random.randn(n, dim)
    fri = np.exp(-damping*time_step)
    noi = np.sqrt((1-fri**2)/(beta*m))
    for t in range(1, size):
        p = position_matrix[t-1]
        v = velocity_matrix[t-1]
        a = acceleration_matrix[t]
        gp = potential_gradient(p, boxsize)
        v_new = v - time_step/(2*m) * gp                     
        p_new = p + time_step/2 *v_new                        
        if pbc:
            p_new = p_new - (p_new > p_max)*(p_max-p_min)
            p_new = p_new + (p_new < p_min)*(p_max-p_min)
        v_new = fri*v_new + noi* R_t                          
        p_new = p_new + time_step/2 *v_new                    
        if pbc:
            p_new = p_new - (p_new > p_max)*(p_max-p_min)
            p_new = p_new + (p_new < p_min)*(p_max-p_min)
        gp = potential_gradient(p_new, boxsize)
        v_new = v_new - time_step/(2*m) * gp                  
        a = - gp/m
        beta = beta + temp*t
        position_matrix[t], velocity_matrix[t], acceleration_matrix[t] = p_new, v_new, a
    return position_matrix, velocity_matrix, acceleration_matrix

In [ ]:
################################### short testing ###########################################
boxsize = (-5, 5)
###### input 2D ######
p_0 = np.array([[1., 0.]])
v_0 = np.array([[0., 0.]])
m = np.array([[1., 1.]])

###### input 3D ######
#p_0 = np.array([[5., 0., 0.], [0., 1., 0.]])
#v_0 = np.array([[0., 0., 0.], [0., 0., 0.]])
#m = np.array([[1., 1., 1.], [1., 1., 1.]])

###### energy conservation ######
res = langevin(potential_gradient, p_0, v_0, m, 100., 0.001, 1., 0.01, 0.01, (-5,5), pbc=False)
#res
#print(a[0][:,0,0])
plt.plot(res[0][:,0, 0], res[1][:,0, 0])

##### kinetic energy ######
#np.dot(res[1][:,0,0],res[0][:,0,1])*2/3==1 #???


## timing

#### loop p = t-1
algorithm where p and v needed to calculate the new p and v equals the last timestep
- total run time :  `%timeit`
     - 

- line time profiling :  `%lprun`
the most time-consuming step is to calculate the next p and v; 
followed by the calculation of the gp/gp_new and riting of the output arrays 
   - ``
        -  
   - ``
        -

on average the loop p=t-1 is 3.8 %Time faster in calculating p_new than p=t+1

### result 
-

In [ ]:
###################################  timing ################################### 
### loop p = t-1 ###
%lprun -f langevin langevin(potential_gradient, p_0, v_0, m, 100., 0.001, 0., 1.0, pbc=True)
%timeit  -n 10 -r 10 langevin(potential_gradient, p_0, v_0, m, 100., 0.001, 0., 1.0, pbc=True)

